<a href="https://colab.research.google.com/github/GauravMayal/fcc_ml/blob/main/Copy_of_fcc_book_recommendation_knn.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

*Note: You are currently reading this using Google Colaboratory which is a cloud-hosted version of Jupyter Notebook. This is a document containing both text cells for documentation and runnable code cells. If you are unfamiliar with Jupyter Notebook, watch this 3-minute introduction before starting this challenge: https://www.youtube.com/watch?v=inN8seMm7UI*

---

In this challenge, you will create a book recommendation algorithm using **K-Nearest Neighbors**.

You will use the [Book-Crossings dataset](http://www2.informatik.uni-freiburg.de/~cziegler/BX/). This dataset contains 1.1 million ratings (scale of 1-10) of 270,000 books by 90,000 users. 

After importing and cleaning the data, use `NearestNeighbors` from `sklearn.neighbors` to develop a model that shows books that are similar to a given book. The Nearest Neighbors algorithm measures distance to determine the “closeness” of instances.

Create a function named `get_recommends` that takes a book title (from the dataset) as an argument and returns a list of 5 similar books with their distances from the book argument.

This code:

`get_recommends("The Queen of the Damned (Vampire Chronicles (Paperback))")`

should return:

```
[
  'The Queen of the Damned (Vampire Chronicles (Paperback))',
  [
    ['Catch 22', 0.793983519077301], 
    ['The Witching Hour (Lives of the Mayfair Witches)', 0.7448656558990479], 
    ['Interview with the Vampire', 0.7345068454742432],
    ['The Tale of the Body Thief (Vampire Chronicles (Paperback))', 0.5376338362693787],
    ['The Vampire Lestat (Vampire Chronicles, Book II)', 0.5178412199020386]
  ]
]
```

Notice that the data returned from `get_recommends()` is a list. The first element in the list is the book title passed in to the function. The second element in the list is a list of five more lists. Each of the five lists contains a recommended book and the distance from the recommended book to the book passed in to the function.

If you graph the dataset (optional), you will notice that most books are not rated frequently. To ensure statistical significance, remove from the dataset users with less than 200 ratings and books with less than 100 ratings.

The first three cells import libraries you may need and the data to use. The final cell is for testing. Write all your code in between those cells.

In [ ]:
# import libraries (you may add additional imports but you may not have to)
import numpy as np
import pandas as pd
from scipy.sparse import csr_matrix
from sklearn.neighbors import NearestNeighbors
import matplotlib.pyplot as plt

In [ ]:
# get data files
!wget https://cdn.freecodecamp.org/project-data/books/book-crossings.zip

!unzip book-crossings.zip

books_filename = 'BX-Books.csv'
ratings_filename = 'BX-Book-Ratings.csv'

--2022-06-01 15:30:25--  https://cdn.freecodecamp.org/project-data/books/book-crossings.zip
Resolving cdn.freecodecamp.org (cdn.freecodecamp.org)... 104.26.3.33, 172.67.70.149, 104.26.2.33, ...
Connecting to cdn.freecodecamp.org (cdn.freecodecamp.org)|104.26.3.33|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 26085508 (25M) [application/zip]
Saving to: ‘book-crossings.zip’

book-crossings.zip  100%[===================>]  24.88M  64.5MB/s    in 0.4s    

2022-06-01 15:30:26 (64.5 MB/s) - ‘book-crossings.zip’ saved [26085508/26085508]

Archive:  book-crossings.zip
  inflating: BX-Book-Ratings.csv     
  inflating: BX-Books.csv            
  inflating: BX-Users.csv            


In [ ]:
# import csv data into dataframes
df_books = pd.read_csv(
    books_filename,
    encoding = "ISO-8859-1",
    sep=";",
    header=0,
    names=['isbn', 'title', 'author'],
    usecols=['isbn', 'title', 'author'],
    dtype={'isbn': 'str', 'title': 'str', 'author': 'str'})

df_ratings = pd.read_csv(
    ratings_filename,
    encoding = "ISO-8859-1",
    sep=";",
    header=0,
    names=['user', 'isbn', 'rating'],
    usecols=['user', 'isbn', 'rating'],
    dtype={'user': 'int32', 'isbn': 'str', 'rating': 'float32'})

In [ ]:
df_books.describe()

,isbn,title,author
count,271379,271379,271378
unique,271379,242154,102042
top,0195153448,Selected Poems,Agatha Christie
freq,1,27,632


In [ ]:
df_ratings.describe()

,user,rating
count,1.149780e+06,1.149780e+06
mean,1.403864e+05,2.866950e+00
std,8.056228e+04,3.842474e+00
min,2.000000e+00,0.000000e+00
25%,7.034500e+04,0.000000e+00
50%,1.410100e+05,0.000000e+00
75%,2.110280e+05,7.000000e+00
max,2.788540e+05,1.000000e+01


In [ ]:
df_ratings.rating.value_counts()

0.0     716109
8.0     103736
10.0     78610
7.0      76457
9.0      67541
5.0      50974
6.0      36924
4.0       8904
3.0       5996
2.0       2759
1.0       1770
Name: rating, dtype: int64

In [ ]:
movies_rating = (df_ratings
                  .set_index("isbn")
                  .join(df_books.set_index("isbn"),
                        how="left"))

movies_rating.head(10)

,user,rating,title,author
isbn,,,,
0330299891,89192,0.0,NaN,NaN
0330299891,181817,6.0,NaN,NaN
0375404120,89192,3.0,NaN,NaN
0375404120,266865,0.0,NaN,NaN
0586045007,89192,0.0,NaN,NaN
9022906116,11676,7.0,NaN,NaN
9022906116,89192,0.0,NaN,NaN
9032803328,89192,0.0,NaN,NaN
9044922564,89192,0.0,NaN,NaN


In [ ]:
movies_rating.loc[movies_rating['title'] == "The Queen of the Damned (Vampire Chronicles (Paperback))"]

,user,rating,title,author
isbn,,,,
0345351525,277478,0.0,The Queen of the Damned (Vampire Chronicles (P...,Anne Rice
0345351525,2033,0.0,The Queen of the Damned (Vampire Chronicles (P...,Anne Rice
0345351525,3827,8.0,The Queen of the Damned (Vampire Chronicles (P...,Anne Rice
0345351525,5609,0.0,The Queen of the Damned (Vampire Chronicles (P...,Anne Rice
0345351525,5772,0.0,The Queen of the Damned (Vampire Chronicles (P...,Anne Rice
...,...,...,...,...
0345351525,274308,0.0,The Queen of the Damned (Vampire Chronicles (P...,Anne Rice
0345351525,275020,7.0,The Queen of the Damned (Vampire Chronicles (P...,Anne Rice
0345351525,275673,7.0,The Queen of the Damned (Vampire Chronicles (P...,Anne Rice


In [ ]:
movies_rating.describe()

,user,rating
count,1.149780e+06,1.149780e+06
mean,1.403864e+05,2.866950e+00
std,8.056228e+04,3.842875e+00
min,2.000000e+00,0.000000e+00
25%,7.034500e+04,0.000000e+00
50%,1.410100e+05,0.000000e+00
75%,2.110280e+05,7.000000e+00
max,2.788540e+05,1.000000e+01


In [ ]:
movies_rating.dropna()

,user,rating,title,author
isbn,,,,
0000913154,171118,8.0,The Way Things Work: An Illustrated Encycloped...,C. van Amerongen (translator)
0001010565,86123,0.0,Mog's Christmas,Judith Kerr
0001010565,209516,0.0,Mog's Christmas,Judith Kerr
0001046438,23902,9.0,Liar,Stephen Fry
0001046713,196149,0.0,Twopence to Cross the Mersey,Helen Forrester
...,...,...,...,...
B000234N76,264317,0.0,Falling Angels,Tracy Chevalier
B000234NC6,100906,0.0,It Must've Been Something I Ate: The Return of...,Jeffrey Steingarten
B00029DGGO,100088,0.0,"Good Wife Strikes Back, The",Elizabeth Buchan


In [ ]:
movies_rating = movies_rating[pd.notnull(movies_rating['title'])]
movies_rating.head()

,user,rating,title,author
isbn,,,,
0000913154,171118,8.0,The Way Things Work: An Illustrated Encycloped...,C. van Amerongen (translator)
0001010565,86123,0.0,Mog's Christmas,Judith Kerr
0001010565,209516,0.0,Mog's Christmas,Judith Kerr
0001046438,23902,9.0,Liar,Stephen Fry
0001046713,196149,0.0,Twopence to Cross the Mersey,Helen Forrester


In [ ]:
condition=movies_rating['rating']>=0
coloumns=["title","rating"]
filtered=movies_rating[condition][coloumns]
filtered.head()



,title,rating
isbn,,
0000913154,The Way Things Work: An Illustrated Encycloped...,8.0
0001010565,Mog's Christmas,0.0
0001010565,Mog's Christmas,0.0
0001046438,Liar,9.0
0001046713,Twopence to Cross the Mersey,0.0


In [ ]:

top_rate_by_decade = (filtered
                     .groupby(["title"])
                     .mean()
                     .sort_values(["rating"],
                                                ascending=False)
                     .groupby(level=0, as_index=False)
                     .apply(lambda x: x.head() if len(x) >= 5 else x.head(1))
                     .reset_index(level=0, drop=True)
                    ).round(2)
                    

In [ ]:
top_rate_by_decade.head()

,rating
title,
"A Light in the Storm: The Civil War Diary of Amelia Martin, Fenwick Island, Delaware, 1861 (Dear America)",2.25
Always Have Popsicles,0.00
Apple Magic (The Collector's series),0.00
"Ask Lily (Young Women of Faith: Lily Series, Book 5)",8.00
Beyond IBM: Leadership Marketing and Finance for the 1990s,0.00


In [ ]:
top_rate_by_decade.value_counts()

rating
0.00      105517
5.00       12922
8.00       11969
10.00      11760
7.00        9357
           ...  
5.32           1
7.27           1
7.21           1
7.18           1
6.06           1
Length: 712, dtype: int64

In [ ]:
top_rate_by_decade.describe()

,rating
count,241090.000000
mean,2.890043
std,3.268494
min,0.000000
25%,0.000000
50%,1.970000
75%,5.000000
max,10.000000


In [ ]:
top_rate_by_decade.loc['The Queen of the Damned (Vampire Chronicles (Paperback))']


rating    2.71
Name: The Queen of the Damned (Vampire Chronicles (Paperback)), dtype: float32

In [ ]:
top_rate_by_decade.loc['The Witching Hour (Lives of the Mayfair Witches)']

rating    3.28
Name: The Witching Hour (Lives of the Mayfair Witches), dtype: float32

In [ ]:
# add your code here - consider creating a new cell for each section of code
df_books.head()

,isbn,title,author
0,0195153448,Classical Mythology,Mark P. O. Morford
1,0002005018,Clara Callan,Richard Bruce Wright
2,0060973129,Decision in Normandy,Carlo D'Este
3,0374157065,Flu: The Story of the Great Influenza Pandemic...,Gina Bari Kolata
4,0393045218,The Mummies of Urumchi,E. J. W. Barber


In [ ]:
df_books.loc[df_books['title'] == "The Queen of the Damned (Vampire Chronicles (Paperback))"]

,isbn,title,author
2527,0345351525,The Queen of the Damned (Vampire Chronicles (P...,Anne Rice
72840,0833563505,The Queen of the Damned (Vampire Chronicles (P...,Anne Rice


In [ ]:
df_books.loc[df_books['title'] == "Catch 22"]

,isbn,title,author
292,0684833395,Catch 22,Joseph Heller
21551,0440204399,Catch 22,Joseph Heller


In [ ]:
df_ratings.head()

,user,isbn,rating
0,276725,034545104X,0.0
1,276726,0155061224,5.0
2,276727,0446520802,0.0
3,276729,052165615X,3.0
4,276729,0521795028,6.0


In [ ]:
df_ratings.loc[df_ratings['isbn'] =="0345351525" ]

,user,isbn,rating
2049,277478,0345351525,0.0
14147,2033,0345351525,0.0
20332,3827,0345351525,8.0
24157,5609,0345351525,0.0
24535,5772,0345351525,0.0
...,...,...,...
1138818,274061,0345351525,0.0
1140813,274308,0345351525,0.0
1144023,275020,0345351525,7.0
1145743,275673,0345351525,7.0


In [ ]:
# function to return recommended books - this will be tested
def get_recommends(book = ""):


  return recommended_books

Use the cell below to test your function. The `test_book_recommendation()` function will inform you if you passed the challenge or need to keep trying.

In [ ]:
books = get_recommends("Where the Heart Is (Oprah's Book Club (Paperback))")
print(books)

def test_book_recommendation():
  test_pass = True
  recommends = get_recommends("Where the Heart Is (Oprah's Book Club (Paperback))")
  if recommends[0] != "Where the Heart Is (Oprah's Book Club (Paperback))":
    test_pass = False
  recommended_books = ["I'll Be Seeing You", 'The Weight of Water', 'The Surgeon', 'I Know This Much Is True']
  recommended_books_dist = [0.8, 0.77, 0.77, 0.77]
  for i in range(2): 
    if recommends[1][i][0] not in recommended_books:
      test_pass = False
    if abs(recommends[1][i][1] - recommended_books_dist[i]) >= 0.05:
      test_pass = False
  if test_pass:
    print("You passed the challenge! 🎉🎉🎉🎉🎉")
  else:
    print("You haven't passed yet. Keep trying!")

test_book_recommendation()